# Tidy Data

Author: [Alexandre Gramfort](http://alexandre.gramfort.net)

<img src="img/borat.png" width=50%>

Based on the work of [J.F. Puget](https://www.ibm.com/developerworks/community/blogs/jfp/entry/Tidy_Data_In_Python?lang=en)

This lecture is based on the article from Hadley Wickham: [Tidy Data](http://vita.had.co.nz/papers/tidy-data.pdf).

If you don't know Hadley Wickham see https://en.wikipedia.org/wiki/Hadley_Wickham

Let's start.

In [ ]:
%matplotlib inline

## Motivation

It is often said that data scientists spend only 20% of their time analyzing their data, and 80% of time cleaning it. Indeed, maintaining a tidy, easy-to-use dataset is crucial for any data analysis especially if it's "big data".

In the paper Tidy Data, Hadley Wickham gives **definitions of tidy and messy data** so that all data scientists can keep their work organized.

In this lecture, you'll learn to transform messy datasets into tidy datasets using the pandas package in python.

We only need two Python packages here.

In [ ]:
import pandas as pd
import numpy as np

Let's show readers which versions we are using.

In [ ]:
pd.__version__

In [ ]:
work = pd.DataFrame([[80., 20.]], columns=['Data cleaning', 'Machine Learning'])

work

In [ ]:
work = work.T
work

In [ ]:
work.columns = ['Work pct.']

In [ ]:
work['Work pct.'].plot(kind='pie')

### Tidy Data and Messy Data

What exactly marks the difference between *tidy* data and *messy* data? It is not only how organized and intuitive the datasets look to our human eyes, but also how **easily and efficiently they can be processed by computers**.

In his seminal paper [Tidy Data](https://www.jstatsoft.org/article/view/v059i10), Hadley Wickham proposed three standards for tidy data:

1. Each variable forms a **column**
2. Each observation forms a **row**
3. Each type of observation forms a **unit**.

Here, we'll focus on the first two rules and show you how we can use the Python package [pandas](http://pandas.pydata.org/) to deal with datasets violating them.

## Introduction

Let's get started, with a dataframe called `messy`.

This dataset, which appears in Wickham's paper, shows the number of people who choose either of two treatments in a hospital.

In [ ]:
messy = pd.DataFrame({'First' : ['John', 'Jane', 'Mary'], 
                      'Last' : ['Smith', 'Doe', 'Johnson'], 
                      'Treatment A' : [np.nan, 16, 3], 
                      'Treatment B' : [2, 11, 1]})
messy

Observe its structure in comparison with Wickham's rules.

This dataset is *messy* because it violates rule #2: it combines Treatment A and Treatment B, two distinct observations, in a single row.

**Messy data sets exist because they are often convenient for showing them to human** as they are compact.
This explains why this form is often used in publications.

People may prefer the transpose view of that data set

In [ ]:
messy.T

**Messy data sets are not that easy to process by statistical or machine learning packages.**

These often assume that examples are provided as rows in a 2d array whose columns are features. This is precisely what a tidy data set is.

Applying the `melt()` function to it creates a tidy version of it.

In [ ]:
messy

In [ ]:
tidy = pd.melt(messy, id_vars=['First','Last'])
tidy

The values are fine but column names aren't really meaningful.  Fortinately, the `melt()` function has arguments for renaming them.

In [ ]:
tidy = pd.melt(messy, id_vars=['First','Last'],
               var_name='treatment', value_name='result')
tidy

## A simple(r) melt example

In [ ]:
messy = pd.DataFrame({'row' : ['A', 'B', 'C'], 
                      'a' : [1, 2, 3],
                      'b' : [4, 5, 6],
                      'c' : [7, 8, 9]})
messy

In [ ]:
pd.melt(messy, id_vars='row')

In [ ]:
tidy = pd.melt(messy, id_vars='row', var_name='dimension',
               value_name='length')
tidy

Pivot is almost the inverse of melt

In [ ]:
messy1 = tidy.pivot(index='row', columns='dimension',
                    values='length')
messy1

This is almost the same as the orginal dataframe, except that row is used as index.  We can move it back to a row easily.

In [ ]:
messy1.reset_index(inplace=True)
messy1

Last step is to remove the name for the set of columns.

In [ ]:
messy1.columns.name = ''
messy1

This is the same as the original dataframe, up to column reordering.

## Column headers are values, not variable names

This is the first issue with messy data in Hadley's paper.  Let's first create the dataframe used as an example.

For practical reasons, it was simpler to first construct the transpose of it.

In [ ]:
messy = pd.DataFrame({'Agnostic' : [27, 34, 60, 81, 76, 137],
                      'Atheist' : [12, 27, 37, 52, 35, 70],
                      'Buddhist' : [27, 21, 30, 34, 33, 58],
                      'Catholic' : [418, 617, 732, 670, 638, 1116],
                      "Don't know/refused" : [15, 14, 15, 11, 10, 35],
                      'Evangelical Prot' : [575, 869, 1064, 982, 881, 1486],
                      'Hindu' : [1, 9, 7, 9, 11, 34],
                      'Historically Black Prot' : [228, 244, 236, 238, 197, 223],
                      "Jehovah's Witness" : [20, 27, 24, 24, 21, 30],
                      'Jewish' : [19, 19, 25, 25, 30, 95],
                     })
    
def transpose(df, columns):
    df = df.T.copy()
    df.reset_index(inplace=True)
    df.columns = columns
    return df

messy = transpose(messy, ['religion', '<$10k', '$10-20k', '$20-30k',
                          '$30-40k', '$40-50k', '$50-75k'])

messy

<div class="alert alert-success">
    <b>QUESTION</b>:
     <ul>
      <li>Why is the dataset messy?</li>
    </ul>
</div>

Again, the `melt()` function is our friend.  We sort the result by religion to make it easier to read.

In [ ]:
tidy = pd.melt(messy, id_vars=['religion'],
               var_name='income', value_name='freq')
tidy.sort_values(by=['religion'], inplace=True)
tidy.head(12)

In [ ]:
tidy.groupby('religion').sum()

## Variables are stored in both rows and columns

This example is a little trickier. This dataset comes from the World Health Organisation, and records the counts of confirmed tuberculosis (TB) cases by country, year, and demographic group.  We first read the input data as a data frame.

This data is available at https://github.com/hadley/tidy-data/blob/master/data/tb.csv

Reading it is easy. We remove the `new_sp_` prefix appearing in most columns, and we rename a couple of columns as well.

In [ ]:
url = "https://raw.githubusercontent.com/hadley/tidy-data/master/data/tb.csv"
tb = pd.read_csv(url)
tb.columns = tb.columns.str.replace('new_sp_','')
tb.rename(columns = {'new_sp' : 'total', 'iso2' : 'country'}, 
          inplace=True)
tb.head(10)

Let's use year 2000, and drop few columns, to stay in sync with Wickham's article.

In [ ]:
messy = tb[tb['year'] == 2000].copy()
messy.drop(['total','m04','m514','f04','f514'], axis=1, inplace=True)
messy.head(10)

In [ ]:
messy.iloc[:, :11].head(10)

The `melt()` function is useful, but is not enough.  Let's use it still.

In [ ]:
molten = pd.melt(messy, id_vars=['country', 'year'], value_name='cases')
molten.sort_values(by=['year', 'country'], inplace=True)
molten.head(10)

In [ ]:
molten['variable'].str.startswith("m")

<div class="alert alert-success">
    <b>QUESTION</b>:
     <ul>
      <li>What is still the problem?</li>
    </ul>
</div>

What isn't really nice is the encoding of sex and age ranges as a string in the `variable` column.

Let's process the dataset to create two additional columns, one for the sex, and one for the age range.

We then remove the `variable` column.

The tidy form also makes it easy to remove the values where the age is `u`.

In [ ]:
def parse_age(s):
    s = s[1:]
    if s == '65':
        return '65+'
    else:
        return s[:-2] + '-' + s[-2:]

tidy = molten[molten['variable'] != 'mu'].copy()
tidy['sex'] = tidy['variable'].apply(lambda s: s[:1])
tidy['age'] = tidy['variable'].apply(parse_age)
tidy = tidy[['country', 'year', 'sex', 'age', 'cases']]
tidy.head(10)

In [ ]:
tidy.groupby("country").sum().\
    sort_values(by='cases', ascending=False)['cases'].head(10).plot(kind='pie')

## Variables are stored in both rows and columns (tricky)

The most **complicated form of messy data occurs when variables are stored in both rows and
columns**. We consider here daily weather data from the Global Historical Climatology Network
for one weather station (MX17004) in Mexico for five months in 2010. It has variables in
individual columns (id, year, month), spread across columns (day, d1–d31) and across rows
(tmin, tmax) (minimum and maximum temperature). Months with less than 31 days have
structural missing values for the last day(s) of the month. The element column is not a
variable; it stores the names of variables.

We'll only consider columns from d1 to d8 to save space.

Let's first create the dataframe.  This time, I create it using an array instead of a dictionary, just for the fun of doing something a bit different.

In [ ]:
columns = ['id', 'year', 'month', 'element', 'd1', 'd2', 'd3', 'd4', 'd5', 'd6', 'd7', 'd8']
data = [['MX17004', 2010, 1, 'tmax', np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, ],
        ['MX17004', 2010, 1, 'tmin', np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, ],
        ['MX17004', 2010, 2, 'tmax', np.nan, 27.3, 24.1, np.nan, np.nan, np.nan, np.nan, np.nan, ],
        ['MX17004', 2010, 2, 'tmin', np.nan, 14.4, 14.4, np.nan, np.nan, np.nan, np.nan, np.nan, ],
        ['MX17004', 2010, 3, 'tmax', np.nan, np.nan, np.nan, np.nan, 32.1, np.nan, np.nan, np.nan, ],
        ['MX17004', 2010, 3, 'tmin', np.nan, np.nan, np.nan, np.nan, 14.2, np.nan, np.nan, np.nan, ],
        ['MX17004', 2010, 4, 'tmax', np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, ],
        ['MX17004', 2010, 4, 'tmin', np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, ],
        ['MX17004', 2010, 5, 'tmax', np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, ],
        ['MX17004', 2010, 5, 'tmin', np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan,]
       ]
messy = pd.DataFrame(data=data, columns=columns); messy

Most of the values are not relevant.  However, filtering the NaN values is imposible here.  We need to melt the dataframe first.

In [ ]:
molten = pd.melt(messy, 
                 id_vars=['id', 'year','month','element',],
                 var_name='day');
molten.dropna(inplace=True)
molten = molten.reset_index(drop=True)
molten

This dataframe is not in tidy form yet.  First, the column `element` contains variable names.  Second, the columns `year, month, day` represent one variable: the date.  Let's fix the latter problem first.

In [ ]:
def f(row):    
    return "%d-%02d-%02d" % (row['year'], row['month'], int(row['day'][1:]))
    
molten['date'] = molten.apply(f, axis=1)
molten = molten[['id', 'element','value','date']].copy()
molten

In [ ]:
molten['date'] = pd.to_datetime(molten['date'])

In [ ]:
molten.info()

In [ ]:
molten

Now we need to pivot the element column.  

In [ ]:
tidy = molten.pivot(index='date',columns='element',values='value')
tidy

In [ ]:
tidy.plot()

Wait a minute.  

Where is the id?

One way to keep it, is to move the id to an index with the `groupby()` function, and apply `pivot()` inside each group. 

In [ ]:
tidy = molten.groupby('id').apply(pd.DataFrame.pivot,
                                  index='date',
                                  columns='element',
                                  values='value')
tidy

We are almost there.  We simply have to move id back as a column with the `reset_index()`.

In [ ]:
tidy.reset_index(inplace=True)
tidy

We get rid of the `element` name.

In [ ]:
tidy.columns.name = ''
tidy

Et Voilà!

## Multiple types in one table

This example is used to illustrate two of the above problems.  

Let's create it. It is an excerpt from the Billboard top hits for 2000.

In [ ]:
columns = ['year','artist','track','time','date entered','wk1','wk2','wk3',]

data = [[2000,"2,Pac","Baby Don't Cry","4:22","2000-02-26",87,82,72,],
        [2000,"2Ge+her","The Hardest Part Of ...","3:15","2000-09-02",91,87,92,],
        [2000,"3 Doors Down","Kryptonite","3:53","2000-04-08",81,70,68,],
        [2000,"98^0","Give Me Just One Nig...","3:24","2000-08-19",51,39,34,],
        [2000,"A*Teens","Dancing Queen","3:44","2000-07-08",97,97,96,],
        [2000,"Aaliyah","I Don't Wanna","4:15","2000-01-29",84,62,51,],
        [2000,"Aaliyah","Try Again","4:03","2000-03-18",59,53,38,],
        [2000,"Adams,Yolanda","Open My Heart","5:30","2000-08-26",76,76,74]
        ]

messy = pd.DataFrame(data=data, columns=columns)
messy

This dataset is messy because there are several observations per row, in the columns wk1, wk2, wk3.

We can get one observation per row by metling the dataset.

In [ ]:
molten = pd.melt(messy, 
                 id_vars=['year','artist','track','time','date entered'],
                 var_name = 'week',
                 value_name = 'rank',
                )
molten.sort_values(by=['date entered','week'], inplace=True)
molten.head()

We can clean the dataset further, first by turning week into number

In [ ]:
molten['week'] = molten['week'].apply(lambda s: int(s[2:]))
molten.head()

Second, we need the starting date of the week for each observation, instead of the date the track entered.

In [ ]:
from datetime import datetime, timedelta

def increment_date(row):
    date = datetime.strptime(row['date entered'], "%Y-%m-%d")
    return date + timedelta(7) * (row['week'] - 1)

molten['date'] = molten.apply(increment_date, axis=1)
molten.drop('date entered', axis=1, inplace=True)
molten.head()

Last, this dataset is **denormalized**.

This is fine for most statistical and machine learning packages, but we might want to normalize it, for example to be stored in an SQL database.

It means that we should **group information that is repeated every week** for a track in a separate table.

This information appears in columns `year ,artist, track, time`.  

In [ ]:
tidy_track = molten[['year','artist','track','time']]\
            .groupby(['year','artist','track'])\
            .first()
tidy_track.reset_index(inplace=True)
tidy_track.reset_index(inplace=True)
tidy_track.rename(columns = {'index':'id'}, inplace=True)
tidy_track

In [ ]:
tidy_rank = pd.merge(molten, tidy_track, on='track')
tidy_rank = tidy_rank[['id', 'date', 'rank']]
tidy_rank.head()

### The end

Making your data tidy will take you a lot of time. Keep it mind that there is
not always a single way of tidying data. It can dependent of the question
to specify what is an observation, a sample, and what is a feature, a variable.